<a href="https://colab.research.google.com/github/chaeyoung97/google-CoLab/blob/master/Kaggle_Titanic%EC%83%9D%EC%A1%B4%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary Classification
- Titanic 탑승자 생존 유무를 예측

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
import numpy as np
import pandas as pd
import random as rnd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from keras import Sequential
from keras.layers import Dense, Activation 

#Data 불러오기 & Data Summary

In [0]:
train = pd.read_csv('/content/drive/My Drive/titanic_data/train.csv')
test = pd.read_csv('/content/drive/My Drive/titanic_data/test.csv')
train_copy = train.copy()
test_id = test.copy()
test_id = test_id[["PassengerId"]]

In [0]:
train.head(2)

In [0]:
test.head(2)

In [0]:
train.describe(include="all")

- Name 의 경우 count == unique 이므로 중복되는 값이 없음
- Age 의경우 count = 714 로 missing value가 있음
- Cabin 의 경우 count = 204 unique= 147 이므로 한 캐비넷을 여러명이서 사용함을 알 수 있음
- Embarked 의 경우 count = 889 로 missing value가 있고,  S를 가장 많이 이용함
- Male 이 이 배에 더 많이 탐


In [0]:
test.describe(include="all")

In [0]:
train.info()

In [0]:
print('train.shape: ', train.shape)
print('test.shape: ', test.shape)

In [0]:
train['Fare']

# Passenger ID 와 Ticket은 필요없음 삭제

In [0]:
train = train.drop(['PassengerId', 'Ticket'], axis = 1)
test = test.drop(['PassengerId', 'Ticket'], axis = 1)

In [0]:
print('train.shape: ', train.shape)
print('test.shape: ', test.shape)

# Missing Value 처리하기

In [0]:
def missing_value(df):
    total = df.isnull().sum().sort_values(ascending=False)
    missing_value = pd.concat([total], axis = 1, keys= ['Missing_Value_Count'])
    return missing_value

In [0]:
missing_value(train)

In [0]:
missing_value(test)

- Cabin은 사용/미사용으로 분류가능함으로 값이 없으면 사용하지 않음= 0 , 값이 있으면 사용함 =1으로 분류
- Age
- Embarked는 가장 많이 이용된 S 로 값을 채우기 가능
- Fare도 중앙값으로 채움

In [0]:
# Cabin 의 missing_value를  X 로 채움
train["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in train['Cabin']])
test["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in test['Cabin']])

In [0]:
train[['Cabin', 'Survived']].groupby(['Cabin'], as_index = False).mean().sort_values(by="Survived", ascending = False)

In [0]:
# Embarked 의 missing_value 를 S로 채움
train['Embarked'].fillna('S', inplace = True)

In [0]:
# Fare 의 missing_Value 를 중앙값으로 채움
test['Fare'].fillna(test['Fare'].median(), inplace = True)

In [0]:
# Age는 여러 특징들을 고려하여 비슷한 특징을 같는 값들의 중앙값으로 채우기
# Pclass Parch Sibsp Fare를 이용하기

def fill_missing_age(df):
    missing_age_idx = list(df["Age"][df['Age'].isnull()].index)

    for i in missing_age_idx:
        median = df["Age"].median()
        predict = df["Age"][(   (df["Pclass"] == df.iloc[i]["Pclass"]) &
                                (df["Parch"] == df.iloc[i]["Parch"]) &
                                (df['SibSp'] == df.iloc[i]["SibSp"]) &
                                (df["Fare"] == df.iloc[i]["Fare"])   )].median()
        if not np.isnan(predict):
            df["Age"].iloc[i] = predict
        else:
            df["Age"].iloc[i] = median
                        

In [0]:
fill_missing_age(train)
fill_missing_age(test)

In [0]:
missing_value(test)

# Data 처리

1.Embarked

In [0]:
def setEmbarked(df):
    df["Embarked"] = df["Embarked"].map ({ "S": 0, "C": 1, "Q": 2}).astype(int)

In [0]:
setEmbarked(train)
setEmbarked(test)

2.Cabin

In [0]:
def setCabin(df):
    df["Cabin"] = df["Cabin"].map(
        {"A":1, "B":1, "C":1, "D":1, "E":1, "F": 1, "G":1, "T":1, "X": 0}).astype(int)

In [0]:
setCabin(train)
setCabin(test)

3.Sex

In [0]:
def setSex(df):
    df["Sex"] = df["Sex"].map({"male":0, "female": 1})

In [0]:
setSex(train)
setSex(test)

4.Name

In [0]:
for i in train_copy:
        train_copy["Title"] = train_copy.Name.str.extract(' ([A-Za-z]+)\.', expand = False) 
pd.crosstab(train_copy['Title'], train_copy['Sex'])


In [0]:

def setName(df):
    for i in df:
        df["Title"] = df.Name.str.extract(' ([A-Za-z]+)\.', expand = False)

    df["Title"] = df["Title"].replace(['Lady', "Countess", "Capt", "Col", "Don", "Dr", "Major","Rev", "Sir", "Jonkheer", "Dona"] , "None")
    df["Title"] = df["Title"].map({"Mr":0, "Miss":1, "Ms":1, "Mme":1, "Mlle":1, "Mrs":1, "Master":2, "None":3})
    df["Title"] = df["Title"].astype(int)


In [0]:
setName(train)
setName(test)
train = train.drop(['Name'], axis = 1)
test = test.drop(['Name'], axis = 1)

In [0]:
pd.crosstab(train['Title'], train['Sex'])

5.Parch & SibSp
- 부모자식관계, 배우자관계가 예측에 영향 줄 수도 있으니 냅둔다
- 가족사이즈를 추가
- 혼자탔는지의 유무도 추가

In [0]:
def familySet(df):
    df["Family"] = df["SibSp"] + df["Parch"] + 1
    df["Alone"] = 0
    df.loc[df["Family"]==1, "Alone"] = 1

In [0]:
familySet(train)
familySet(test)

In [0]:
# SibSp > Parch인 경우 SibSp < Parch인 경우보다 생존확률이 더 낮아짐
train[['Family',"SibSp","Parch", 'Survived']].groupby(['Family'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [0]:
train.head()

6.Age


In [0]:
train['Age_Range'] = pd.cut(train['Age'], 8)

In [0]:
train[['Age_Range', 'Survived']].groupby(['Age_Range'], as_index=False).mean().sort_values(by='Age_Range', ascending=True)


In [0]:
def setAge(df):  
    df.loc[ df['Age'] <= 10.368, 'Age'] = 0
    df.loc[(df['Age'] > 10.368) & (df['Age'] <= 20.315), 'Age'] = 1
    df.loc[(df['Age'] > 20.315) & (df['Age'] <= 30.263), 'Age'] = 2
    df.loc[(df['Age'] > 30.263) & (df['Age'] <= 40.21), 'Age'] = 3
    df.loc[(df['Age'] > 40.21) & (df['Age'] <= 50.158), 'Age'] = 4
    df.loc[(df['Age'] > 50.158) & (df['Age'] <= 60.105), 'Age'] = 5
    df.loc[(df['Age'] >60.105) & (df['Age'] <= 70.052), 'Age'] = 6
    df.loc[ df['Age'] > 70.052, 'Age'] = 7
    

In [0]:
setAge(train)
setAge(test)

In [0]:
train=train.drop(['Age_Range'], axis =1)


In [0]:
train[['Age', 'Survived']].groupby(['Age'], as_index=False).mean().sort_values(by='Age', ascending=True)


# Train Data 분리

In [0]:
x_train = train.drop("Survived", axis=1)
y_train = train["Survived"]

In [0]:
print('x_train.shape :', x_train.shape)
print('test.shape :', test.shape)

# Model 만들기

In [0]:
from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

kfold = StratifiedKFold(n_splits=10)

random_state = 2
classifiers = []
classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(LinearDiscriminantAnalysis())

cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier, x_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC","DecisionTree","AdaBoost",
"RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","KNeighboors","LogisticRegression","LinearDiscriminantAnalysis"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")

########################################

# DNN 모델

In [0]:
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Activation
import keras
from keras.layers import Dropout ,BatchNormalization
from keras.optimizers import Adam , SGD
from keras import optimizers


In [0]:
#가장 성능 좋은 모델
model = Sequential()
model.add(Dense(units=32, activation= 'relu', input_dim =11 ))
model.add(Dense(units= 16, activation= "relu"))
model.add(Dense(units=1, activation ='sigmoid'))
sgd=optimizers.SGD(lr=0.001)

model.compile(optimizer=sgd, loss = "binary_crossentropy", metrics = ['accuracy'])


In [0]:
history = model.fit(x_train, y_train, epochs = 1000)

In [0]:
prec = model.predict(test)

In [0]:
prec = pd.DataFrame(prec)

In [0]:
prec

In [0]:


prec.loc[prec[0] >=0.5] =1
prec.loc[prec[0]<0.5] = 0

prec = prec[0].astype(int)

In [0]:
submission = pd.DataFrame({
            "PassengerId": test_id["PassengerId"],
        "Survived": prec
    })
submission.to_csv("drive/My Drive/titanic9.csv", index=False)

# VotingClassifier

In [0]:

m1 = LogisticRegression(solver="liblinear", random_state= 42)
m2 = DecisionTreeClassifier(max_depth = 3, criterion = "gini", random_state= 0)
m3 = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
m4 = RandomForestClassifier(max_depth = 10, n_estimators=100, random_state=42)
m5 = SVC(probability = True, gamma="auto", C = 10, random_state=42)
m6 = GaussianNB()
m9 = SGDClassifier(loss = "log")

In [0]:
vot = VotingClassifier(estimators=[('1', m1), ('2', m4), ('3', m5), ('4', m2), ('5',m3),('6', m6),
                                    ('9',m9)], voting = 'soft')


In [0]:
vot.fit(x_train, y_train)

In [0]:

pre = vot.predict(test)


In [0]:
pre = pd.DataFrame(pre)

In [0]:
pre = pre[0].astype(int)

In [0]:
pre

In [0]:
submission = pd.DataFrame({
            "PassengerId": test_id["PassengerId"],
        "Survived": pre
    })
submission.to_csv("drive/My Drive/titanic7.csv", index=False)